In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,5,6'
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/data/lyz/qwen/Qwen2-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/root/miniconda3/envs/qwen2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.37s/it]


In [3]:
#原始gsm8k数据读取
import json
import jsonlines
import re
from data_process import read_gsm8k
file = '/data/lyz/datas/gsm8k/train.jsonl'
datas = []
datas = read_gsm8k(file)


In [ ]:
max_gen = 10
for i,data in enumerate(datas):
    question = data['question']
    final_answer = data['final_answer']
    cnt = 0 
    flag = 1 
    while flag and cnt < max_gen:
        system_prompt = "You are a helpful assistant. You need to help me to solve the math problem step by step. And then return the final answer follow the rule."
        instruct = "You should solve the math problem step by step. And at the end of the output, you should give your final numerical answer use \'So, the numerical answer is: \'.\n"
        gen_prompt = "Let's think step by step.\n [Step 1]:"
        prompt = instruct + '<question>: '+ question + gen_prompt
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(i)
        # print(response)
        # print('final answer: ',final_answer)
        data['steps'] = response
        # print('######################')
        cnt += 1 
        flag = 0
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [1]:
output_file = '/data/lyz/math_dpo/output/gsm_zs_1.json'
with open(output_file,'w')as f:
    json.dump(datas,f,indent= 4)

#


In [ ]:
openai_api_key = "empty"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [21]:
import time
def generate(data,client):
    question = data['question']
    final_answer = data['final_answer']
    system_prompt = "You are a helpful assistant. You need to help me to solve the math problem step by step."
    instruct = "You should solve the math problem step by step. And at the end of the output, you should give your final numerical answer use \'So, the numerical answer is: \'.\n"
    gen_prompt = "Let's think step by step.\n [Step 1]:"
    prompt = instruct + '<question>: '+ question + gen_prompt
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    print(len(prompt.split(' ')))
    chat_response = client.chat.completions.create(
    model="/data/lyz/qwen/Qwen2-7B-Instruct",
    messages=messages,
    temperature=0.7,
    top_p=0.8,
    max_tokens=256,
    extra_body={
        "repetition_penalty": 1.05,
        },
    )
    return chat_response.choices[0].message.content
start = time.time()
res = generate(datas[0],client)
end = time.time()
print(res)
print(end - start)

67
To find out how many clips Natalia sold in April and May altogether, we first need to determine how many clips she sold in May.

Given:
- Natalia sold 48 clips in April.
- She sold half as many clips in May as she did in April.

[Step 2]: Calculate the number of clips sold in May.
Since she sold half as many clips in May as in April, we can calculate the number of clips sold in May by dividing the number of clips sold in April by 2.

Number of clips sold in May = Number of clips sold in April / 2
Number of clips sold in May = 48 / 2

[Step 3]: Perform the calculation.
Number of clips sold in May = 24

[Step 4]: Calculate the total number of clips sold in April and May.
Total number of clips sold = Number of clips sold in April + Number of clips sold in May
Total number of clips sold = 48 + 24

[Step 5]: Perform the addition.
Total number of clips sold = 72

So, the numerical answer is: Natalia sold a total of 72 clips altogether in April and May.
2.0289952754974365


In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'
from vllm import LLM, SamplingParams
llm = LLM(model="/data/lyz/qwen/Qwen2-7B-Instruct", trust_remote_code=True, tensor_parallel_size=4,tokenizer_mode="auto", dtype="auto")



def print_outputs(outputs):
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    print("-" * 80)


/root/miniconda3/envs/qwen2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-30 14:10:09,785	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 10-30 14:10:10 config.py:899] Defaulting to use mp for distributed inference
INFO 10-30 14:10:10 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='/data/lyz/qwen/Qwen2-7B-Instruct', speculative_config=None, tokenizer='/data/lyz/qwen/Qwen2-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/data/ly

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  3.37it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.97it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.84it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.00it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.99it/s]


INFO 10-30 14:10:19 model_runner.py:1025] Loading model weights took 3.5546 GB
(VllmWorkerProcess pid=3579438) (VllmWorkerProcess pid=3579439) INFO 10-30 14:10:19 model_runner.py:1025] Loading model weights took 3.5546 GB
INFO 10-30 14:10:19 model_runner.py:1025] Loading model weights took 3.5546 GB
(VllmWorkerProcess pid=3579440) INFO 10-30 14:10:19 model_runner.py:1025] Loading model weights took 3.5546 GB


INFO 10-30 14:10:22 distributed_gpu_executor.py:57] # GPU blocks: 297353, # CPU blocks: 18724
(VllmWorkerProcess pid=3579440) INFO 10-30 14:10:27 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=3579440) INFO 10-30 14:10:27 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-30 14:10:28 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-30 14:10:28 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB

In [2]:
sampling_params = SamplingParams(temperature=0.9, top_p=0.9,max_tokens=256)

In [6]:
#原始gsm8k数据读取
import json
import jsonlines
import re
from data_process import read_gsm8k
file = '/data/lyz/datas/gsm8k/train.jsonl'
datas = []
datas = read_gsm8k(file)


In [7]:
import time 
batch = []
batch_size = 64
start = time.time()
for data in datas:
    question = data['question']
    final_answer = data['final_answer']
    system_prompt = "You are a helpful assistant. You need to help me to solve the math problem step by step."
    instruct = "You should solve the math problem step by step. And at the end of the output, you should give your final numerical answer use \'So, the numerical answer is: \'.\n"
    gen_prompt = "Let's think step by step.\n [Step 1]:"
    prompt = instruct + '<question>: '+ question + gen_prompt
    message = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    batch.append(message)
    if len(batch) == batch_size:
        outputs = llm.chat(batch,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        print('#',len(outputs))
        end = time.time()
        print(end - start)
        print_outputs(outputs)
        start = time.time()

# 64
3.558438539505005
Prompt: "<|im_start|>system\nYou are a helpful assistant. You need to help me to solve the math problem step by step.<|im_end|>\n<|im_start|>user\nYou should solve the math problem step by step. And at the end of the output, you should give your final numerical answer use 'So, the numerical answer is: '.\n<question>: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?Let's think step by step.\n [Step 1]:<|im_end|>\n<|im_start|>assistant\n", Generated text: "To find out how many clips Natalia sold in May, we need to take half of the amount she sold in April since it's given that she sold half as many clips in May compared to April.\n\nNatalia sold 48 clips in April.\n\n[Step 2]: \n\nTo find the number of clips sold in May, we divide the number of clips sold in April by 2:\n\nClips sold in May = 48 / 2\n\n[Step 3]:\n\nNow, let's calculate the total number of clip

In [ ]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Generated text: {generated_text!r}")

Generated text: "To find out how many clips Natalia sold in May, we need to take half of the amount she sold in April since it's given that she sold half as many clips in May compared to April.\n\nNatalia sold 48 clips in April.\n\n[Step 2]: \n\nTo find the number of clips sold in May, we divide the number of clips sold in April by 2:\n\nClips sold in May = 48 / 2\n\n[Step 3]:\n\nNow, let's calculate the total number of clips sold by Natalia in both April and May:\n\nTotal clips sold = Clips sold in April + Clips sold in May\n\nLet's compute these values:\nClips sold in May = 48 / 2 = 24\n\nTotal clips sold = 48 (April) + 24 (May) = 72\n\nSo, the numerical answer is: \n\nNatalia sold a total of 72 clips altogether in April and May."
Generated text: "First, we need to convert the babysitting time from minutes to hours because Weng's hourly rate is given in dollars per hour. We know that 60 minutes make up 1 hour. \n\nWeng babysat for 50 minutes, so we convert this time to hours:\n\\[ \\

In [4]:
conversation = [
    {
        "role": "system",
        "content": "You are a helpful assistant"
    },
    {
        "role": "user",
        "content": "Hello"
    },
]
conversations = [conversation for _ in range(10)]

# # We turn on tqdm progress bar to verify it's indeed running batch inference
outputs = llm.chat(messages=conversations,
                   sampling_params=sampling_params,
                   use_tqdm=True)


Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 10/10 [00:01<00:00,  5.15it/s, est. speed input: 97.79 toks/s, output: 51.47 toks/s]


In [69]:
import re
def extract_numerical_answer(solutions):
    answers = []
    for solution in solutions:
        # 使用正则表达式提取 "So, the numerical answer is:" 后的数字
        match = re.search(r"So, the numerical answer is:(.*)", solution, re.IGNORECASE | re.DOTALL)

        if match:
            # 提取出这部分文本内容
            answer_line = match.group(1).strip()
            
            # 从中提取数字（包括整数和小数）
            num_match = re.findall(r"[-+]?\d+(?:,\d{3})*(?:\.\d+)?", answer_line)

            if num_match:
                final_answer_str = num_match[-1].replace(',', '').replace(' ', '')
                try:
                    # 转换为浮点数或整数
                    final_answer = float(final_answer_str) if '.' in final_answer_str else int(final_answer_str)
                    answers.append(final_answer)
                except ValueError:
                    print('error')
                    answers.append(None)  # 如果转换失败设为 None
            else:
                # print(solution)
                answers.append(None)
        else:
            answers.append(None)  # 如果没有找到匹配的答案也设为 None
    return answers


In [40]:
def eval_answer(answers,target):
    labels = []
    for ans in answers:
        if not ans:
            labels.append(False)
        else:
            labels.append(float(ans) == target)
    return labels

In [80]:
import json
file = '/data/lyz/math_dpo/output/gsm8k_gen_naive.json'
with open(file) as f:
    datas = json.load(f)
cnt = 0
for i,data in enumerate(datas):
    solutions = data['solutions']
    target = data['final_answer']
    answers = extract_numerical_answer(solutions)
    labels = eval_answer(answers,target)
    print(i)
    print(target)
    print(answers)
    print(labels)
    for lab in labels:
        if lab :
            cnt += 1
            break 
    # if labels[0]:
    #     cnt += 1 
            
print('acc: ',cnt / len(datas))

0
72.0
[72, 72, None, 72, 72, 72, 72, 72, 72, 72]
[True, True, False, True, True, True, True, True, True, True]
1
10.0
[10, 50, 50, 50, 50, 50, 10, 50, 10, 50]
[True, False, False, False, False, False, True, False, True, False]
2
5.0
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[True, True, True, True, True, True, True, True, True, True]
3
42.0
[42, 42, 42, 42, 42, 42, 42, 42, 42, 42]
[True, True, True, True, True, True, True, True, True, True]
4
624.0
[624, 624, 624, 624, 624, 624, 624, 624, 624, 624]
[True, True, True, True, True, True, True, True, True, True]
5
35.0
[35, 35, 35, 35, None, 35, 35, 35, 35, 35]
[True, True, True, True, False, True, True, True, True, True]
6
48.0
[48, 48, 48, 48, 48, 48, 48, 48, 48, 48]
[True, True, True, True, True, True, True, True, True, True]
7
16.0
[16, None, None, None, 16, None, None, None, None, None]
[True, False, False, False, True, False, False, False, False, False]
8
41.0
[None, 41, 41, 41, 41, 41, None, None, 41, 41]
[False, True, True, True, True, True,

In [74]:
sol = 'So, the numerical answer is: 82085208'
match = re.search(r"So, the numerical answer is:(.*)", sol, re.IGNORECASE | re.DOTALL)

if match:
    # 提取出这部分文本内容
    answer_line = match.group(1).strip()
    
    # 从中提取数字（包括整数和小数）

    num_match = re.findall(r"[-+]?\d+(?:,\d{3})*(?:\.\d+)?", answer_line)

    if num_match:
        print(num_match)
        final_answer_str = num_match[-1].replace(',', '')
        final_answer = float(final_answer_str) if '.' in final_answer_str else int(final_answer_str)
        print('##########:',final_answer)

['82085208']
##########: 82085208
